<img src="../imgs/Adevinta-ULPGC-logo.jpg" width="530px" align="right">

# **Tutorial de Pytorch 11: Transfer Learning: ResNet**

**ResNet**, que significa *Residual Networks*, es un tipo de arquitectura de red neuronal convolutiva (CNN) introducida en 2015 en el trabajo "Deep Residual Learning for Image Recognition" de *Microsoft Research*. La innovación clave de ResNet es su enfoque en el aprendizaje de funciones residuales utilizando conexiones de salto (skip connections) para facilitar el flujo de gradientes a través de la red, lo que permite entrenar redes mucho más profundas sin caer en problemas de desvanecimiento o explosión del gradiente.

Las redes ResNet están compuestas por bloques residuales, donde la entrada de cada bloque se pasa a través de algunas capas convolutivas (por lo general, dos o tres) y luego se suma al inicio del bloque antes de aplicar una función de activación. Esta suma de la entrada del bloque con la salida de las capas convolutivas permite que la red aprenda modificaciones incrementales respecto a la entrada, en lugar de aprender la representación completa desde cero en cada capa. Teóricamente, esto facilita el aprendizaje de la red, ya que solo necesita aprender las diferencias (residuos) que necesita agregar a la entrada para llegar a la salida correcta.

ResNet demostró ser muy efectivo, ganando la competición **ILSVRC 2015** (ImageNet Large Scale Visual Recognition Challenge) y estableciendo nuevos estándares para modelos de visión por computadora. Las versiones más comunes de ResNet incluyen ResNet-18, ResNet-34, ResNet-50, ResNet-101 y ResNet-152, donde el número indica la profundidad de la red en términos de capas. La introducción de ResNet permitió el entrenamiento práctico de redes mucho más profundas, algunas con cientos o incluso miles de capas, promoviendo avances significativos en varias tareas de visión por computadora.

<div align="center">
    <img src="./imgs/resnet.png" >
    <p style="text-align: center;color:gray">Arquitectura de ResNet-34</p>
</div>

## **Datos**

Preparamos los datos igual que en el modelo anterior.

In [5]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.nn import Dropout

# Establecemos semilla
torch.manual_seed(42)

# Definimos las transformaciones que deseas aplicar a las imágenes
transform = transforms.Compose([
    transforms.ToTensor(),           # Convierte las imágenes a tensores
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # Normaliza las imágenes para VGG16
])

# Ruta donde se encuentran las imágenes
data_dir = "./data/house_rooms"

# Creamos un objeto ImageFolder
dataset = ImageFolder(root=data_dir, transform=transform)

# Dividimos el conjunto de datos en entrenamiento (80%) y validación (20%)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Definimos el tamaño del lote y creamos los DataLoader
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


## **Modelo**

In [6]:
# Cargamos el modelo ResNet50 preentrenado
resnet = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)

# Congela los pesos de las capas del modelo
for param in resnet.parameters():
    param.requires_grad = False

# Descongela las últimas capas del modelo
for param in resnet.layer4.parameters():
    param.requires_grad = True

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /Users/cayetano/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:03<00:00, 32.5MB/s]


In [8]:
resnet

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [9]:
resnet.fc = nn.Sequential(
    nn.Linear(2048, 1024),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(1024, 5)
)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = resnet.to(device)

## **Entrenamiento**

In [10]:
# Definimos la función de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entrenamiento del modelo
epochs = 50
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}")

# Validación del modelo
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on validation set: {100 * correct / total}%")

Epoch 1/50, Loss: 0.5644878397385279
Epoch 2/50, Loss: 0.16542210057377815
Epoch 3/50, Loss: 0.07529591311785308
Epoch 4/50, Loss: 0.04133121065696645
Epoch 5/50, Loss: 0.0329691340607083
Epoch 6/50, Loss: 0.05034090584376827
Epoch 7/50, Loss: 0.04345490204904116
Epoch 8/50, Loss: 0.03370260718017534
Epoch 9/50, Loss: 0.02113008867000286
Epoch 10/50, Loss: 0.015419524009551176
Epoch 11/50, Loss: 0.06877011650287772
Epoch 12/50, Loss: 0.0195124687866844
Epoch 13/50, Loss: 0.022742150204783575
Epoch 14/50, Loss: 0.01688630695394805
Epoch 15/50, Loss: 0.011060522157067404
Epoch 16/50, Loss: 0.014331782707096974
Epoch 17/50, Loss: 0.029159051758059915
Epoch 18/50, Loss: 0.030269512510241122
Epoch 19/50, Loss: 0.009358657558495355
Epoch 20/50, Loss: 0.008444863043471494
Epoch 21/50, Loss: 0.006766664904882253
Epoch 22/50, Loss: 0.00793997023147037
Epoch 23/50, Loss: 0.007318352209756885
Epoch 24/50, Loss: 0.008914383971965766
Epoch 25/50, Loss: 0.008313140786922855
Epoch 26/50, Loss: 0.0073